## Install Crew

In [11]:
!uv add crewai crewai_tools duckduckgo-search


    █████▒██▀███  ▓█████ ▄▄▄       ██ ▄█▀     Hostname: Freak-PC 
  ▓██   ▒▓██ ▒ ██▒▓█   ▀▒████▄     ██▄█▒     
  ▒████ ░▓██ ░▄█ ▒▒███  ▒██  ▀█▄  ▓███▄░      OS: EndeavourOS 
  ░▓█▒  ░▒██▀▀█▄  ▒▓█  ▄░██▄▄▄▄██ ▓██ █▄     
  ░▒█░   ░██▓ ▒██▒░▒████▒▓█   ▓██▒▒██▒ █▄     User: Freak 
  ▒ ░   ░ ▒▓ ░▒▓░░░ ▒░ ░▒▒   ▓▒█░▒ ▒▒ ▓▒     
   ░       ░▒ ░ ▒░ ░ ░  ░ ▒   ▒▒ ░░ ░▒ ▒░     Date: 10 June 2025 
   ░ ░     ░░   ░    ░    ░   ▒   ░ ░░ ░     
            ░        ░  ░     ░  ░░  ░       

Resolved 281 packages in 1ms
Audited 269 packages in 0.11ms


### Import Required

In [12]:
from crewai import Agent, Crew, Process, Task, LLM

### Create Custom Tools for Duck Duck Go Searcg

In [13]:
from crewai.tools import BaseTool
from langchain_community.tools import DuckDuckGoSearchRun
from pydantic import BaseModel, Field

class DDGSearchInput(BaseModel):
    """Defines the Input schema for Duck Duck Go Tool"""
    query: str = Field("Input query for searching the web using Duck Duck Go Tool")

class DDGSearchTool(BaseTool):
    name: str = "DuckDuckGo Search Tool"
    description: str = "Search the web for a given query."
    args_schema: type[BaseModel] = DDGSearchInput

    def _run(self, query: str) -> str:
        duckduckgo_tool = DuckDuckGoSearchRun()
        response = duckduckgo_tool.invoke(query)
        return response

### Create Crew Agents

In [14]:
trending_company_finder = Agent(
        role="Indian Financial News Analyst that finds trending companies registered in Indian Stock Exchange NSE / BSE from {sector}",
        goal="""You read the latest Indian news, then find 2-3 companies in Indian Stock Exchange NSE/BSE that are trending in the news for further research.
            Always pick new companies. Don't pick the same company twice.""",
        backstory="""You are a market expert with a knack for picking out the most interesting companies based on latest news.
            You spot multiple companies that are trending in the news.""",
        verbose=True,
        llm=LLM(model="ollama/qwen3:1.7b", base_url="http://localhost:11434"),
        tools=[DDGSearchTool()],
    )

financial_researcher = Agent(
        role="Senior Financial Researcher",
        goal="""Given details of trending companies in the news, you provide comprehensive analysis of each in a report.""",
        backstory="""You are a financial expert with a proven track record of deeply analyzing hot companies and building comprehensive reports.""",
        verbose=True,
        llm=LLM(model="ollama/qwen3:1.7b", base_url="http://localhost:11434"),
        tools=[DDGSearchTool()]
    )

stock_picker = Agent(
        role="Stock Picker from Research",
        goal="""Given a list of researched companies with investment potential, you select the best one for investment,
            notifying the user and then providing a detailed report. Don't pick the same company twice.""",
        backstory="""You're a meticulous, skilled financial analyst with a proven track record of equity selection.
            You have a talent for synthesizing research and picking the best company for investment.""",
        verbose=True,
        llm=LLM(model="ollama/qwen3:1.7b", base_url="http://localhost:11434"),
    )

manager = Agent(
        role="Manager",
        goal="""You are a skilled project manager who can delegate tasks in order to achieve your goal, which is to pick the best company for investment.""",
        backstory="""You are an experienced and highly effective project manager who can delegate tasks to the right people.""",
        verbose=True,
        llm=LLM(model="ollama/qwen3:1.7b", base_url="http://localhost:11434"),
        allow_delegation=True,
    )

### Structured Output Models

In [15]:
from pydantic import BaseModel, Field
from typing import List

class TrendingCompany(BaseModel):
    """ A company that is in the news and attracting attention """
    name: str = Field(description="Company name")
    ticker: str = Field(description="Stock ticker symbol")
    reason: str = Field(description="Reason this company is trending in the news")

class TrendingCompanyList(BaseModel):
    """ List of multiple trending companies that are in the news """
    companies: List[TrendingCompany] = Field(description="List of companies trending in the news")

class TrendingCompanyResearch(BaseModel):
    """ Detailed research on a company """
    name: str = Field(description="Company name")
    market_position: str = Field(description="Current market position and competitive analysis")
    future_outlook: str = Field(description="Future outlook and growth prospects")
    investment_potential: str = Field(description="Investment potential and suitability for investment")

class TrendingCompanyResearchList(BaseModel):
    """ A list of detailed research on all the companies """
    research_list: List[TrendingCompanyResearch] = Field(description="Comprehensive research on all trending companies")

### Create Crew Tasks

In [16]:
find_trending_companies = Task(
        description="""Find the top trending companies in the news in {sector} by searching the latest news. Find new companies that you've not found before.""",
        expected_output="""A list of trending companies in {sector}""",
        agent=trending_company_finder,
        output_pydantic=TrendingCompanyList,
        output_file="output/trending-companies.json"
    )

research_trending_companies = Task(
        description="""Given a list of trending companies, provide detailed analysis of each company in a report by searching online""",
        expected_output="""A report containing detailed analysis of each company""",
        agent=financial_researcher,
        output_pydantic=TrendingCompanyResearchList,
        output_file="output/trending-companies-research.json",
        context=[find_trending_companies]
    )

pick_best_company = Task(
        description="""Analyze the research findings and pick the best company for investment.
            Then respond with a detailed report on why you chose this company, and which companies were not selected.""",
        expected_output="""The chosen company and why it was chosen; the companies that were not selected and why they were not selected.""",
        agent=stock_picker,
        output_file="output/picked-stock-decision.md",
        context=[research_trending_companies]
    )

### Create Crew

In [17]:
crew =  Crew(
        agents=[trending_company_finder, financial_researcher, stock_picker],
        tasks=[find_trending_companies, research_trending_companies, pick_best_company],
        process=Process.hierarchical,
        verbose=True,
        manager_agent=manager
    )

### Run Crew

In [18]:
from IPython.display import Markdown
inputs = {
    'sector': 'Finance',
}

result = crew.kickoff(inputs=inputs)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c23c5d1a-c0dc-4f70-92c7-a620e83cb059                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: d23d5f91-95f4-447d-840f-799977ac525a
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: d23d5f91-95f4-447d-840f-799977ac525a
       Status: Executing Task...
    └── 🤖 Agent: Manager
            Status: In Progress

# Agent: Manager
## Task: Find the top trending companies in the news in Finance by searching the latest news. Find new companies that you've not found before.


🤖 Agent: Manager
    Status: In Progress

...



# Agent: Manager
## Final Answer: 
Sunbit is the most suitable investment candidate. Its blockchain and DeFi initiatives align with high-growth sectors, offer technological leadership, and position it to capitalize on regulatory clarity and institutional adoption. The analysis confirms its strong market position, future outlook, and investment potential, despite risks.




🚀 Crew: crew
├── 📋 Task: d23d5f91-95f4-447d-840f-799977ac525a
│      Assigned to: Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Manager
│           Status: ✅ Completed
├── 📋 Task: f9e6f627-6d28-4cc4-bd20-c36dd21613a3
│      Assigned to: Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Manager
│           Status: ✅ Completed
└── 📋 Task: 3a6e6e3f-c6e4-4cc7-94cb-22ecfbf6fb99
       Status: Executing Task...
    ├── 🤖 Agent: Manager
    │       Status: In Progress
    └── 🤖 Agent: Manager
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: d23d5f91-95f4-447d-840f-799977ac525a
│      Assigned to: Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Manager
│           Status: ✅ Completed
├── 📋 Task: f9e6f627-6d28-4cc4-bd20-c36dd21613a3
│      Assigned to: Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Manager
│           Status: ✅ Completed
└── 📋 Task: 3a6e6e3f-c6e4-4cc7-94cb-22ecfbf6fb99
       Assigned to: Manager
       Status: ✅ Completed
    ├── 🤖 Agent: Manager
    │       Status: In Progress
    └── 🤖 Agent: Manager
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3a6e6e3f-c6e4-4cc7-94cb-22ecfbf6fb99                                                                     │
│  Agent: Manager                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c23c5d1a-c0dc-4f70-92c7-a620e83cb059                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [19]:
Markdown(result.raw)

Sunbit is the most suitable investment candidate. Its blockchain and DeFi initiatives align with high-growth sectors, offer technological leadership, and position it to capitalize on regulatory clarity and institutional adoption. The analysis confirms its strong market position, future outlook, and investment potential, despite risks.